Step 1: Explore the Sandbox Data

In [ ]:
import os
print(os.getcwd())

D:\Projects_2025\ml_service


In [2]:
# In your notebook
import pandas as pd
df_sample = pd.read_csv('data/raw/accepted_2007_to_2018Q4.csv', nrows=50000)

C:\Users\21658\AppData\Local\Temp\ipykernel_20748\1147157911.py:3: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sample = pd.read_csv('data/raw/accepted_2007_to_2018Q4.csv', nrows=50000)


Step 2: Initial Exploration

In [3]:
# See the first 5 rows to get a visual feel for the data
df_sample.head()

# Get a summary of all columns, their data types, and non-null counts
# This is one of the most important commands.
df_sample.info(verbose=True, show_counts=True)

# See the percentage of missing values for each column, sorted
print(df_sample.isnull().mean().sort_values(ascending=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 151 columns):
 #    Column                                      Non-Null Count  Dtype  
---   ------                                      --------------  -----  
 0    id                                          50000 non-null  int64  
 1    member_id                                   0 non-null      float64
 2    loan_amnt                                   50000 non-null  float64
 3    funded_amnt                                 50000 non-null  float64
 4    funded_amnt_inv                             50000 non-null  float64
 5    term                                        50000 non-null  object 
 6    int_rate                                    50000 non-null  float64
 7    installment                                 50000 non-null  float64
 8    grade                                       50000 non-null  object 
 9    sub_grade                                   50000 non-null  object 
 1

Step 3 : Feature Selection & Dropping Columns

In [4]:
# --- Feature Selection & Dropping Columns ---

# 1. Columns that are mostly or completely empty
# We identify these from your .info() output (e.g., member_id, sec_app..., hardship...)
cols_to_drop_empty = [
    'member_id', 'revol_bal_joint', 'sec_app_fico_range_low', 'sec_app_fico_range_high',
    'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc',
    'sec_app_open_acc', 'sec_app_revol_util', 'sec_app_open_act_il', 'sec_app_num_rev_accts',
    'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med',
    'sec_app_mths_since_last_major_derog', 'hardship_type', 'hardship_reason', 'hardship_status',
    'deferral_term', 'hardship_amount', 'hardship_start_date', 'hardship_end_date',
    'payment_plan_start_date', 'hardship_length', 'hardship_dpd', 'hardship_loan_status',
    'orig_projected_additional_accrued_interest', 'hardship_payoff_balance_amount',
    'hardship_last_payment_amount', 'desc', 'mths_since_last_record', 'mths_since_last_major_derog',
    'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'next_pymnt_d'
]

# 2. "Leaky" columns that contain information from the future (after the loan is issued)
leaky_columns = [
    'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
    'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
    'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
    'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low',
    'debt_settlement_flag', 'debt_settlement_flag_date', 'settlement_status',
    'settlement_date', 'settlement_amount', 'settlement_percentage', 'settlement_term'
]

# 3. Unnecessary or redundant columns (IDs, free text, etc.)
unnecessary_cols = [
    'id', 'url', 'emp_title', 'zip_code', 'policy_code',
    'funded_amnt', # Usually identical to loan_amnt
    'funded_amnt_inv', # Usually identical to loan_amnt
    'sub_grade' # Grade is a less granular version that is often sufficient
]

# Combine all columns to drop into a single list
all_cols_to_drop = cols_to_drop_empty + leaky_columns + unnecessary_cols

# Use a set to handle any duplicates, then convert back to a list
all_cols_to_drop = list(set(all_cols_to_drop))

# Drop the columns from the DataFrame
df_sample_cleaned = df_sample.drop(columns=all_cols_to_drop, errors='ignore')

print("Column cleanup complete!")
print(f"Original number of columns: {len(df_sample.columns)}")
print(f"Number of columns after dropping: {len(df_sample_cleaned.columns)}")

Column cleanup complete!
Original number of columns: 151
Number of columns after dropping: 87


In [5]:
df_sample_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 87 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   loan_amnt                       50000 non-null  float64
 1   term                            50000 non-null  object 
 2   int_rate                        50000 non-null  float64
 3   installment                     50000 non-null  float64
 4   grade                           50000 non-null  object 
 5   emp_length                      47011 non-null  object 
 6   home_ownership                  50000 non-null  object 
 7   annual_inc                      50000 non-null  float64
 8   verification_status             50000 non-null  object 
 9   issue_d                         50000 non-null  object 
 10  loan_status                     50000 non-null  object 
 11  pymnt_plan                      50000 non-null  object 
 12  purpose                         

In [6]:
import numpy as np

# Make sure you are using your cleaned DataFrame from the previous step
# df_sample_cleaned = ...

# --- Step 1: Define the function to map loan_status to a binary target ---
def map_loan_status(status):
    if pd.isna(status):
        return np.nan
    status = str(status).lower()
    
    # Define what a "Bad Loan" is
    bad_loan_statuses = ['charged off', 'default', 'does not meet the credit policy. charged off']
    
    # Define what a "Good Loan" is
    good_loan_statuses = ['fully paid', 'does not meet the credit policy. fully paid']
    
    if any(st in status for st in bad_loan_statuses):
        return 1  # High Risk
    elif any(st in status for st in good_loan_statuses):
        return 0  # Low Risk
    else:
        # This will catch 'Current', 'In Grace Period', etc.
        return np.nan

# --- Step 2: Apply the function and create the target column ---
df_sample_cleaned['target'] = df_sample_cleaned['loan_status'].apply(map_loan_status)

# --- Step 3: Remove rows where the outcome is not yet final ---
# The .dropna() function will remove all rows where 'target' is NaN
df_sample_cleaned.dropna(subset=['target'], inplace=True)

# Optional: Drop the original 'loan_status' column as we now have 'target'
df_sample_cleaned.drop(columns=['loan_status'], inplace=True)

# --- Step 4: Check the results ---
print("Target variable 'target' created.")
print("Loans with non-final outcomes have been removed.")
print("\nClass distribution in your data sample:")
print(df_sample_cleaned['target'].value_counts())

Target variable 'target' created.
Loans with non-final outcomes have been removed.

Class distribution in your data sample:
target
0.0    34978
1.0     9028
Name: count, dtype: int64


Part A: Clean the Numerical Columns

In [7]:
import numpy as np

# 1. Identify all numerical columns (excluding our 'target' variable)
numerical_cols = df_sample_cleaned.select_dtypes(include=np.number).columns.tolist()
if 'target' in numerical_cols:
    numerical_cols.remove('target')

# 2. Loop through each numerical column and fill missing values with the median
print("Filling missing values in numerical columns...")
for col in numerical_cols:
    median_val = df_sample_cleaned[col].median()
    df_sample_cleaned[col].fillna(median_val, inplace=True)

print("Numerical columns cleaned.")

Filling missing values in numerical columns...
Numerical columns cleaned.


Part B: Convert Categorical Columns to Numbers

In [9]:
# 1. Identify all remaining categorical columns (they have the 'object' dtype)
categorical_cols = df_sample_cleaned.select_dtypes(include='object').columns.tolist()

print(f"\nApplying one-hot encoding to categorical columns: {categorical_cols}")

# 2. Apply one-hot encoding
# drop_first=True is a good practice to avoid redundant columns
df_final_sample = pd.get_dummies(df_sample_cleaned, columns=categorical_cols, drop_first=True, dtype=float)

print("Categorical columns converted.")


Applying one-hot encoding to categorical columns: ['term', 'grade', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'pymnt_plan', 'purpose', 'title', 'addr_state', 'earliest_cr_line', 'initial_list_status', 'application_type', 'hardship_flag', 'disbursement_method']
Categorical columns converted.


Final Verification

In [10]:
# Check for any remaining missing values (the output should be 0)
print("\nTotal remaining missing values:", df_final_sample.isnull().sum().sum())

# Check the data types of a few columns (they should all be numbers like float64, int64, or bool)
print("\nFinal DataFrame info:")
df_final_sample.info()


Total remaining missing values: 0

Final DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 44006 entries, 0 to 49999
Columns: 765 entries, loan_amnt to application_type_Joint App
dtypes: float64(765)
memory usage: 257.2 MB
